<a href="https://colab.research.google.com/github/venkatesh010196/Flower-classification-assignment/blob/main/flower-classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
train_file_path = '/content/drive/MyDrive/Colab Notebooks/archive/train'
val_file_path = '/content/drive/MyDrive/Colab Notebooks/archive/val'

In [3]:
import pandas as pd
import numpy as np


In [4]:
import os
import pandas as pd
import numpy as np
from torchvision.io import read_image
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset, DataLoader

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

# Define the dataset and data loader for 10-fold stratified cross-validation
def create_stratified_data_loader(img_dir, annotations_file, transform, batch_size, num_folds=10):
    # Load your dataset
    dataset = CustomImageDataset(annotations_file, img_dir, transform=transform)

    # Get labels and indices
    labels = dataset.img_labels['label'].values
    indices = np.arange(len(labels))

    # Initialize StratifiedKFold
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

    data_loaders = []

    for train_index, val_index in skf.split(indices, labels):
        train_dataset = torch.utils.data.Subset(dataset, train_index)
        val_dataset = torch.utils.data.Subset(dataset, val_index)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        data_loaders.append((train_loader, val_loader))

    return data_loaders

# Example usage
img_dir = 'path_to_your_images'
annotations_file = 'path_to_your_annotations_file.csv'
transform = your_transform_function  # Define your image transformation function
batch_size = 32

data_loaders = create_stratified_data_loader(img_dir, annotations_file, transform, batch_size)

# Iterate over the 10 folds
for fold, (train_loader, val_loader) in enumerate(data_loaders):
    print(f"Fold {fold + 1}")
    # Train your model on train_loader and validate on val_loader for each fold



NameError: ignored